In [1]:
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
import torch.nn.functional as F

training_data = torchvision.datasets.MNIST(
    root = 'data',
    download=True,
    train=True,
    transform=torchvision.transforms.Compose(
        [torchvision.transforms.Resize(28), torchvision.transforms.ToTensor()]
    ),
)

test_data = torchvision.datasets.MNIST(
    root = 'data',
    download=True,
    train=False,
    transform=torchvision.transforms.Compose(
        [torchvision.transforms.Resize(28), torchvision.transforms.ToTensor()]
    ),
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(training_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        # self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=0)
        # self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=0)

        # self.linear_relu_stack = nn.Sequential(
        #     nn.Linear(32 * 5 * 5, 10),
        #     nn.ReLU(),
        # )

        self.fc1 = nn.Linear(784, 250)
        self.fc2 = nn.Linear(250, 100)
        self.fc3 = nn.Linear(100, 10)

    def forward(self, x):
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

model = NeuralNetwork()

In [2]:
learning_rate = 0.001
batch_size = 64
epochs = 20

In [3]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model.to(device)

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss for backprop
        pred = model(X.to(device))
        loss = loss_fn(pred, y.to(device))

        # Backpropagation by setting grad to zero, calculating using backprop engine and stepping (using learning rate)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0
    
    # No gradient on training data (faster computation and no optimization happening here anyway)
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X.to(device))
            test_loss += loss_fn(pred, y.to(device)).item()
            correct += (pred.argmax(1) == y.to(device)).type(torch.float).sum().item()

    test_loss /= size
    correct /= size
    
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.312243  [    0/60000]
loss: 2.315523  [ 6400/60000]
loss: 2.299329  [12800/60000]
loss: 2.298895  [19200/60000]
loss: 2.284414  [25600/60000]
loss: 2.295447  [32000/60000]
loss: 2.287126  [38400/60000]
loss: 2.288369  [44800/60000]
loss: 2.280816  [51200/60000]
loss: 2.270606  [57600/60000]
Test Error: 
 Accuracy: 26.5%, Avg loss: 0.035549 

Epoch 2
-------------------------------
loss: 2.280362  [    0/60000]
loss: 2.280044  [ 6400/60000]
loss: 2.268474  [12800/60000]
loss: 2.259965  [19200/60000]
loss: 2.248578  [25600/60000]
loss: 2.259679  [32000/60000]
loss: 2.244786  [38400/60000]
loss: 2.255860  [44800/60000]
loss: 2.239950  [51200/60000]
loss: 2.226243  [57600/60000]
Test Error: 
 Accuracy: 43.6%, Avg loss: 0.034859 

Epoch 3
-------------------------------
loss: 2.234100  [    0/60000]
loss: 2.229480  [ 6400/60000]
loss: 2.224132  [12800/60000]
loss: 2.198693  [19200/60000]
loss: 2.196171  [25600/60000]
loss: 2.205376  [32000/600